In [1]:
import random
import re
from typing import Any

import pandas as pd
from dash import dcc, html
from rich.pretty import install, pprint

In [2]:
install(indent_guides=False, expand_all=True)

### Refactor `update_sidebar_style` Function to Handle Any Number of Pages

In [ ]:
num_pages = 7

paths = [f"/{path}" for path in range(num_pages)]
input_icon_src = {
    path.replace("/", ""): f"/assets/images{path}_light.svg" for path in paths
}
input_link_class = {
    path.replace(
        "/", ""
    ): "px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700"
    for path in paths
}
pathname = random.choice(paths)

available_paths = paths.copy()
available_paths.remove(pathname)

COLORS = {
    "bg_color_prefix": "bg",
    "bg_color_dark": "bg-slate-800",
    "bg_color_light": "bg-emerald-50",
    "text_color_prefix": "text",
    "text_color_dark": "text-slate-800",
    "text_color_light": "text-emerald-50",
    "hover_color_dark": "hover:bg-slate-700",
}

In [11]:
pathname

'/0'

In [12]:
input_icon_src


{
    '0': '/assets/images/0_light.svg',
    '1': '/assets/images/1_light.svg',
    '2': '/assets/images/2_light.svg',
    '3': '/assets/images/3_light.svg',
    '4': '/assets/images/4_light.svg',
    '5': '/assets/images/5_light.svg',
    '6': '/assets/images/6_light.svg'
}

In [13]:
input_link_class


{
    '0': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700',
    '1': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700',
    '2': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700',
    '3': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700',
    '4': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700',
    '5': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700',
    '6': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700'
}

In [15]:
pprint(available_paths)

['/1', '/2', '/3', '/4', '/5', '/6']

In [3]:
def update_utility_classes(
    current_classes: str,
    remove_classes: list[str] | None = None,
    add_classes: list[str] | None = None,
    ignore_prefix_warning: bool = False,
) -> str:
    """Update a utility class string by removing and/or adding classes.

    Removes/adds utility classes from/to the `current_classes` string. At least one of
    the arguments `remove_classes` or `add_classes` must be provided. These arguments
    are given in the form of a list of strings representing the classes to be removed or
    added.

    If an incoming add class has a prefix that matches a class(es) in the
    `current_classes` string, a warning will be shown but the add will still be
    completed. The warning is there to make it known that the addition of the class
    could override these prefix matched existing classes in the `current_classes`
    string. The developer should evaluate the add and ensure that this is not the case.
    It is possible that there is no overriding behavior even when there are multiple
    classes with the same prefix. If this is the case, the warning can be suppressed by
    setting the `ignore_prefix_warning` to True.

    Parameters
    ----------
    current_classes : str
        Current utility class string.
    remove_classes : list[str] | None, optional
        Classes to be removed, by default None.
    add_classes : list[str] | None, optional
        Classes to be added, by default None.
    ignore_prefix_warning : bool, optional
        Flag for suppressing the prefix warning, by default False.

    Returns
    -------
    str
        Updated utility class string.
    """
    # Check if both optional arguments were not provided.
    if remove_classes is None and add_classes is None:
        raise RuntimeError(
            "Both arguments `remove_classes` and `add_classes` were not provided. "
            "Please provide at least one of these arguments."
        )

    current_class_list = current_classes.split()
    # Pattern to match the prefix of a utility class.
    prefix_pattern = r"^-?([a-z:]+)"

    # Remove and/or Add Classes --------------------------------------------------------
    if remove_classes:
        for remove_class in remove_classes:
            try:
                current_class_list.remove(remove_class)
            except ValueError as err:
                raise ValueError(
                    f"The string '{remove_class}', from the `remove_classes` argument, "
                    f"was not found in the `current_classes` string:\n"
                    f"'{current_classes}'"
                ) from err

    if add_classes:
        for add_class in add_classes:
            # Check if the class is already in the `current_class_list`.
            if add_class in current_class_list:
                raise RuntimeError(
                    f"The string '{add_class}', from the `add_classes` argument, is "
                    f"already found within the `current_classes` string:\n"
                    f"'{current_classes}'"
                )

            # Capture the prefix of the incoming add class.
            try:
                match = re.search(prefix_pattern, add_class)
                assert match is not None
                prefix = match.group(1)
            except AttributeError as err:
                raise RuntimeError(
                    f"The string '{add_class}', from the `add_classes` argument, is "
                    f"not a valid utility class."
                ) from err

            # Capture all classes from `current_classes_list` that match the prefix for
            # the current add class.
            prefix_match_classes = []
            for util_class in current_class_list:
                prefix_match = re.search(prefix, util_class)
                if prefix_match:
                    prefix_match_classes.append(util_class)

            # Warn that there are matches that could result in classes that get
            # overridden by the add. This is a warning and not an error because it is
            # possible to have more than one utility class with the same prefix and not
            # have any overriding behavior.
            if prefix_match_classes:
                if not ignore_prefix_warning:
                    print(
                        f"WARNING: Upon adding the string '{add_class}', the following "
                        f"class(es) with the same prefix '{prefix}' were found within "
                        f"the `current_classes` string: {prefix_match_classes}\nIf "
                        f"this addition does not result in conflicts, this warning can "
                        f"be suppressed by setting the `ignore_prefix_warning` "
                        f"argument to True.\n"
                    )
            current_class_list.append(add_class)
    return " ".join(current_class_list)

In [4]:
def click_new_page(pathname, available_paths):
    """Simulate a click on a page link."""
    choice = random.choice(available_paths)
    available_paths.append(pathname)
    available_paths.remove(choice)

    return choice, available_paths

In [5]:
def update_sidebar_style(pathname, input_icon_src, input_link_class):
    """Update icons and link colors when a link is active.

    Parameters
    ----------
    pathname : str
        Current pathname of the app.
    input_icon_src : dict[str, str]
        Contains the src attribute for each page link's icon.
    input_link_class : dict[str, str]
        Contains the class attribute for each page link.

    Returns
    -------
    dict[str, dict[str, str]]
        Contains the updated icon src attributes and page link class attributes.
    """
    icon_src = input_icon_src.copy()
    link_class = input_link_class.copy()

    # Reset previously active page link to styling for inactive state ------------------
    iis = pd.Series(link_class)
    pattern_active = COLORS["bg_color_light"]
    is_active = iis[iis.str.contains(pattern_active)]

    # If there is a page with a dark icon, change it back to light icon.
    if not is_active.empty:
        previous_pathname = is_active.index[0]
        icon_src[previous_pathname] = re.sub(
            r"_dark\.", "_light.", icon_src[previous_pathname]
        )
        link_class[previous_pathname] = update_utility_classes(
            current_classes=is_active.loc[previous_pathname],
            remove_classes=[COLORS["bg_color_light"], COLORS["text_color_dark"]],
            add_classes=[
                COLORS["bg_color_dark"],
                COLORS["text_color_light"],
                COLORS["hover_color_dark"],
            ],
        )
    else:
        previous_pathname = None

    # Update current active page link to styling for active state ----------------------
    if pathname == "/":
        page = "home"
    else:
        page = pathname.replace("/", "")

    # Try/except is used because it is possible to enter a route that doesn't exist,
    # resulting in a KeyError when trying to access that `page` name.
    try:
        icon_src[page] = re.sub(r"_light\.", "_dark.", icon_src[page])
        link_class[page] = update_utility_classes(
            current_classes=link_class[page],
            remove_classes=[
                COLORS["bg_color_dark"],
                COLORS["text_color_light"],
                COLORS["hover_color_dark"],
            ],
            add_classes=[COLORS["bg_color_light"], COLORS["text_color_dark"]],
        )
    except KeyError:
        print(f"The page route '{page}' does not exist.")

    return {
        "output_icon_src": {**icon_src},
        "output_link_class": {**link_class},
    }

In [6]:
print("Load Page")
print(f"starting pathname: {pathname}")
print(f"starting input_icon_src: {input_icon_src}")
print(f"starting input_link_class: {input_link_class}")
print("On Load, Callback Fires")
output = update_sidebar_style(pathname, input_icon_src, input_link_class)
print(f"output_icon_src: {output['output_icon_src']}")
print(f"output_link_class: {output['output_link_class']}")
print("------------------------")
for _ in range(5):
    print("Click On New Page")
    pathname, available_paths = click_new_page(pathname, available_paths)
    print(f"pathname: {pathname}")
    print("Callback Fires")
    output = update_sidebar_style(
        pathname, output["output_icon_src"], output["output_link_class"]
    )
    print(f"output_icon_src: {output['output_icon_src']}")
    print(f"output_link_class: {output['output_link_class']}")
    print("------------------------")
print("")

Load Page
starting pathname: /4
starting input_icon_src: {'0': '/assets/images/0_light.svg', '1': '/assets/images/1_light.svg', '2': '/assets/images/2_light.svg', '3': '/assets/images/3_light.svg', '4': '/assets/images/4_light.svg', '5': '/assets/images/5_light.svg', '6': '/assets/images/6_light.svg'}
starting input_link_class: {'0': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700', '1': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700', '2': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700', '3': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700', '4': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700', '5': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700', '6': 'px-4 py-2 flex space-x-2 items-center bg-slate-800 text-emerald-50 hover:bg-slate-700'}
On Load, Ca

### Reconfigure Sidebar Layout With Language Sections

In [4]:
# Stand in page_registry variable
page_registry: dict[str, dict[str, Any]] = {
    "pages.home": {
        "path": "/",
        "name": "Home",
        "sidebar": True,
        "language": "about",
        "id_link": "home_link",
        "relative_path": "/",
    },
    "pages.background": {
        "path": "/background",
        "name": "Handbell Music Validation",
        "sidebar": True,
        "language": "javascript",
        "id_link": "background_link",
        "relative_path": "/background",
    },
    "pages.dashboard": {
        "path": "/dashboard",
        "name": "Dashboard",
        "sidebar": True,
        "language": "python",
        "id_link": "dashboard_link",
        "relative_path": "/dashboard",
    },
    "pages.mlb_the_show": {
        "path": "/mlb_the_show",
        "name": "MLB The Show",
        "sidebar": True,
        "language": "python",
        "id_link": "dashboard_link",
        "relative_path": "/mlb_the_show",
    },
    "pages.another": {
        "path": "/another",
        "name": "Another",
        "sidebar": True,
        "language": "sql",
        "id_link": "another_link",
        "relative_path": "/another",
    },
    "pages.not_found_404": {
        "path": "/not_found_404",
        "name": "Not Found 404",
        # "sidebar": True,
        "language": "not_found_404",
        "id_link": "not_found_404_link",
        "relative_path": "/not-found-404",
    },
}

In [13]:
language_sections = {}
for page in page_registry.values():
    if page.get("sidebar"):
        language = page["language"]
        language_title = language.title() if language != "sql" else language.upper()
        language_heading = html.Div(
            language_title, className="px-1 py-2 text-slate-600 text-sm font-semibold"
        )

        link = dcc.Link(
            page["name"],
            id=page["id_link"],
            href=page["relative_path"],
            className="px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all",
        )

        if language not in language_sections:
            language_sections[language] = {"heading": language_heading, "links": []}
        language_sections[language]["links"].append(link)

pages = []
for language, section_data in language_sections.items():
    link_div = html.Div(section_data["links"], className="LINK DIV")
    section_div = html.Div([section_data["heading"], link_div], className="SECTION DIV")
    pages.append(section_div)

print("language_sections")
pprint(language_sections, indent_guides=False, expand_all=True)
print("pages")
pprint(pages, indent_guides=False, expand_all=True)

language_sections


{
    'about': {
        'heading': Div(children='About', className='px-1 py-2 text-slate-600 text-sm font-semibold'),
        'links': [
            Link(children='Home', href='/', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='home_link')
        ]
    },
    'javascript': {
        'heading': Div(children='Javascript', className='px-1 py-2 text-slate-600 text-sm font-semibold'),
        'links': [
            Link(children='Handbell Music Validation', href='/background', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='background_link')
        ]
    },
    'python': {
        'heading': Div(children='Python', className='px-1 py-2 text-slate-600 text-sm font-semibold'),
        'links': [
            Link(children='Dashboard', href='/dashboard', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='dashboard_link'),
            Link(children='MLB The Show', href='/mlb_the_show', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='dashboard_link')
        ]
    },
    'sql': {
        'heading': Div(children='SQL', className='px-1 py-2 text-slate-600 text-sm font-semibold'),
        'links': [
            Link(children='Another', href='/another', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='another_link')
        ]
    }
}

pages


[
    Div(children=[Div(children='About', className='px-1 py-2 text-slate-600 text-sm font-semibold'), Div(children=[Link(children='Home', href='/', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='home_link')], className='LINK DIV')], className='SECTION DIV'),
    Div(children=[Div(children='Javascript', className='px-1 py-2 text-slate-600 text-sm font-semibold'), Div(children=[Link(children='Handbell Music Validation', href='/background', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='background_link')], className='LINK DIV')], className='SECTION DIV'),
    Div(children=[Div(children='Python', className='px-1 py-2 text-slate-600 text-sm font-semibold'), Div(children=[Link(children='Dashboard', href='/dashboard', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='dashboard_link'), Link(children='MLB The Show', href='/mlb_the_show', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='dashboard_link')], className='LINK DIV')], className='SECTION DIV'),
    Div(children=[Div(children='SQL', className='px-1 py-2 text-slate-600 text-sm font-semibold'), Div(children=[Link(children='Another', href='/another', className='px-2 py-2 text-center font-semibold text-xs text-emerald-50 hover:bg-slate-700 hover:pl-2.5 hover:pr-1.5 transition-all', id='another_link')], className='LINK DIV')], className='SECTION DIV')
]

In [8]:
PAGE_METADATA = ("home", "background", "handbell_music", "dashboard", "another")

IDS = {
    "header": {page: {"link": f"header_{page}_link"} for page in PAGE_METADATA},
    "sidebar": {page: {"link": f"sidebar_{page}_link"} for page in PAGE_METADATA},
}
IDS


{
    'header': {
        'home': {
            'link': 'header_home_link'
        },
        'background': {
            'link': 'header_background_link'
        },
        'handbell_music': {
            'link': 'header_handbell_music_link'
        },
        'dashboard': {
            'link': 'header_dashboard_link'
        },
        'another': {
            'link': 'header_another_link'
        }
    },
    'sidebar': {
        'home': {
            'link': 'sidebar_home_link'
        },
        'background': {
            'link': 'sidebar_background_link'
        },
        'handbell_music': {
            'link': 'sidebar_handbell_music_link'
        },
        'dashboard': {
            'link': 'sidebar_dashboard_link'
        },
        'another': {
            'link': 'sidebar_another_link'
        }
    }
}